# Network Analysis Project

## Assignment Description

You will have to deliver your project at the end of the course (deadline to be determined). You have to hand in a presentation (in PDF or Power Point format). It is mandatory to include the following information:

- Basic network description of your data (what type of network it is, what does it represent, is it real or synthetically generated, etc). In practice, the result of project phase #1 (finding data).
- Basic network statistics of your data (number of nodes, edges, clustering, degree distribution, etc). In practice, the result of project phase #2 (exploratory data analysis).
- A clear statement of your research question, the result of project phase #3.
- The analysis, results, and interpretation that allow you to answer your research question, the result of project phase #4.
You're free to include this in the order you prefer and to add any additional information you deem necessary, but these are the mandatory components.

The format of the oral is as follows: the students mAlaskae a joint presentation followed by group questions. Subsequently the students are having individual examination with additional questions while the rest of the group is outside the room. The length of the oral will be 15 minutes X number of group members plus one -- for instance, a group of 6 will have 105 minutes ((6+1)*15). Which means you have 15 minutes of group exam plus 15 minutes of individual exam each.

## Environment Setup

This project uses Python libraries that are essential for the performed analysis. MAlaskae sure to have the dependencies listed in requirements.txt installed locally using the Python Package Manager pip.

In [1]:
# %%capture
# %pip install -r requirements.txt

### Packages

In [2]:
# network representation and algorithms
import networkx as nx
from networkx.algorithms import bipartite
from pyvis.network import Network
from networkx import linalg as nxla
import powerlaw as pl                                            # powerlaw fits for degree distribution
from IPython.display import display, Image, Markdown             # display images and markdown in jupyter

# general data science libraries
from matplotlib import pyplot as plt                            # basic plotting
import seaborn as sns                                           # advanced plotting
import numpy as np                                              # for representing n-dimensional arrays
import scipy as sp                                              # numerical computation
import pandas as pd                                             # dataframes

# python standard library
from time import time                                           # used for timing execution
from datetime import date, datetime                             # get current data and time
import json                                                     # read/ write json
import re                                                       # regex search 
import os                                                       # os operations
import random                                                   # randomness
from collections import Counter                                 # efficient counting
import contextlib

# custom imports
from cscripts import metrics
from cscripts import plotting
from cscripts import summarise
from cscripts import backboning # michele
#from cscripts import github_api
from cscripts import projections

from cscripts.airport_metadata import gen_airport_metadata

c:\Users\jackh\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\__init__.py:177: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


### Set global style of plots

Below you can specify global style for all plots or any other setups related to plots visualization.

In [3]:
sns.set_style("darkgrid")
sns.set(rc={"xtick.bottom" : True, "ytick.left" : True})

### Flags

Flags are used to control the run flow of the notebook when executed at once. This is useful, to prevent operations that should only produce a result once, from running multiple times.

In [4]:
# section flags
LOAD_DATA = True # Loads raw data for initial inspection
TRANSFORM_DATA = False # Transforms raw data into more suitable format (Load data needs to be on as well)
COMPUTE_PROJECTIONS = False
GENERATE_SUMMARY_PROJ = False # Summary related to projections only
GET_ONE_CC = True # Do you want to get one connected component for each projection?
DO_BACKBONING = False
GENERATE_SUMMARY_PROJ_BACKB = False # Summary related to projected AND backboned graph
SAVE_FIG = False # Do you want to save all generated figures?
RANDOM_SAMPLE = False

### Constants

In [5]:
PATH_TO = {}
PATH_TO['data'] = {}
PATH_TO['data']['raw'] = 'data/raw'
PATH_TO['data']['transformed'] = 'data/transformed'
PATH_TO['data']['projections'] = 'data/projections'
PATH_TO['data']['backboning'] = 'data/backboning'
PATH_TO['data']['figures'] = 'data/figures'
PATH_TO['data']['summaries'] = 'data/graph_summaries'
PATH_TO['data']['metadata'] = 'data/metadata'

### Folder Structure

Create relevant folders to read from and write to, if not yet existent.

In [6]:
# iterate over path_to dict
for path in PATH_TO['data'].values():
    os.mAlaskaedirs(path) if not os.path.exists(path) else print('Already exists.')

Already exists.
Already exists.
Already exists.
Already exists.
Already exists.
Already exists.
Already exists.


#### #01 Loading and Inspecting Raw Data

In [7]:
if LOAD_DATA:
    airports_raw = pd.read_csv(f"{PATH_TO['data']['raw']}/out.opsahl-usairport", sep=' ', skiprows=2, header=None, names=['from_node', 'to_node', 'weight'])
    airports_iata_code = pd.read_csv(f"{PATH_TO['data']['raw']}/airport_code.txt", sep=' ', header=None, names=['id', 'code'])

# the number of unique airports in the dataset, that is the number of unique from_node and to_node
# combine from_node and to_node into one list
n_airports = set(airports_raw['from_node'].tolist() + airports_raw['to_node'].tolist())
print(f"Number of unique airports: {len(n_airports)}")

# Display first few rows of each DataFrame
airports_raw.head(), airports_iata_code.head()

Number of unique airports: 1574


(   from_node  to_node  weight
 0          1        2       1
 1          1        3       7
 2          4        5       3
 3          6        7       5
 4          6        8   14137,
    id code
 0   1  06A
 1   2  08A
 2   3  1G4
 3   4  6B0
 4   5  8F3)

In [8]:
from cscripts.airport_metadata import gen_airport_metadata

airport_metadata_df = gen_airport_metadata(airports_iata_code)
airport_metadata_df.head()

,id,code,name,city,country,subd,elevation,lat,lon,tz
0,1,06A,Moton Field Municipal Airport,Tuskegee,US,Alabama,264.0,32.460472,-85.680028,America/Chicago
1,2,08A,Wetumpka Municipal Airport,Wetumpka,US,Alabama,197.1,32.527333,-86.331028,America/Chicago
2,3,1G4,Grand Canyon West Airport,Peach Springs,US,Arizona,4816.7,35.986111,-113.816917,America/Phoenix
3,4,6B0,Middlebury State Airport,Middlebury,US,Vermont,494.1,43.986472,-73.095552,America/New_York
4,5,8F3,Crosbyton Municipal Airport,Crosbyton,US,Texas,3018.1,33.623833,-101.240861,America/Chicago


In [9]:
# Get unique airports IDs from airports_raw
unique_airports = set(airports_raw['from_node'].tolist() + airports_raw['to_node'].tolist())

# Check if all unique airport IDs are in airports_code_code
all_airports_in_iata_code = all(airport in airports_iata_code['id'].tolist() for airport in unique_airports)

# print(len(unique_airports), all_airports_in_iata_code, len(airports_iata_code_filtered))

# Remove rows in airports_iata_code where id is not in unique_airports
airports_iata_code_filtered = airports_iata_code[airports_iata_code['id'].isin(unique_airports)]

print(airports_iata_code_filtered)

        id code
0        1  06A
1        2  08A
2        3  1G4
3        4  6B0
4        5  8F3
...    ...  ...
1569  1570  TCT
1570  1571  TDF
1571  1572  TDZ
1572  1573  TEB
1573  1574  TEK

[1574 rows x 2 columns]


In [10]:
import airportsdata

# 5. Load airport metadata
airport_metadata = airportsdata.load()

# what type is airport_metadata? is it a list, dict, dataframe?
print(type(airport_metadata))

# print first 10 airports
for airport in list(airport_metadata.values())[:10]:
    print(airport)

# how many airports are there in the dataset?
print(f"Number of airports in the dataset: {len(airport_metadata)}")

# how many US airports are there in the dataset?
print(f"Number of US airports in the dataset: {len([airport for airport in airport_metadata.values() if airport['country'] == 'US'])}")

# is there a airport with lid == '06A'? If so, print it
print(f"Is there a airport with lid == '06A'? {[airport for airport in airport_metadata.values() if airport['lid'] == '06A']}")

# is there a airport with lid == 'A02' or iata == 'A02' or faa == 'A02'? If so, print it
print(f"Is there a airport with lid == 'A03' or iata == 'A03'? {[airport for airport in airport_metadata.values() if airport['lid'] == 'A27' or airport['iata'] == '1G4']}")

<class 'dict'>
{'icao': '00AA', 'iata': '', 'name': 'Aero B Ranch Airport', 'city': 'Leoti', 'subd': 'Kansas', 'country': 'US', 'elevation': 3435.0, 'lat': 38.7040222222222, 'lon': -101.473911111111, 'tz': 'America/Chicago', 'lid': '00AA'}
{'icao': '00AK', 'iata': '', 'name': 'Lowell Field', 'city': 'Anchor Point', 'subd': 'Alaska', 'country': 'US', 'elevation': 252.0, 'lat': 59.9488888888889, 'lon': -151.692222222222, 'tz': 'America/Anchorage', 'lid': '00AK'}
{'icao': '00AL', 'iata': '', 'name': 'Epps Airpark', 'city': 'Harvest', 'subd': 'Alabama', 'country': 'US', 'elevation': 820.0, 'lat': 34.8648102777778, 'lon': -86.7702775, 'tz': 'America/Chicago', 'lid': '00AL'}
{'icao': '00AR', 'iata': '', 'name': 'Arland Airport', 'city': 'Bennington', 'subd': 'Kansas', 'country': 'US', 'elevation': 1352.0, 'lat': 38.9696510833333, 'lon': -97.6015566111111, 'tz': 'America/Chicago', 'lid': '00AR'}
{'icao': '00AS', 'iata': '', 'name': 'Fulton Airport', 'city': 'Alex', 'subd': 'Oklahoma', 'countr

LID code key

In [11]:
filtered_airport_metadata_lid = {}

for icao, airport_info in airport_metadata.items():
    lid = airport_info['lid']
    country = airport_info['country']
    if lid != '': # only keep airports with lid code
        airport_info.pop('icao', None)  # Remove the 'icao' field
        filtered_airport_metadata_lid[lid] = airport_info

# Print the first 10 entries of the filtered dictionary
for lid, airport_info in list(filtered_airport_metadata_lid.items())[:10]:
    print(lid, airport_info)

# how many airports are there in the filtered_airport_metadata_lid?
n_airports = len(filtered_airport_metadata_lid)
print(f"Number of airports: {n_airports}")

# how many has country 'US'?
n_us_airports = sum(airport_info['country'] == 'US' for airport_info in filtered_airport_metadata_lid.values())
print(f"Number of US airports: {n_us_airports}")

00AA {'iata': '', 'name': 'Aero B Ranch Airport', 'city': 'Leoti', 'subd': 'Kansas', 'country': 'US', 'elevation': 3435.0, 'lat': 38.7040222222222, 'lon': -101.473911111111, 'tz': 'America/Chicago', 'lid': '00AA'}
00AK {'iata': '', 'name': 'Lowell Field', 'city': 'Anchor Point', 'subd': 'Alaska', 'country': 'US', 'elevation': 252.0, 'lat': 59.9488888888889, 'lon': -151.692222222222, 'tz': 'America/Anchorage', 'lid': '00AK'}
00AL {'iata': '', 'name': 'Epps Airpark', 'city': 'Harvest', 'subd': 'Alabama', 'country': 'US', 'elevation': 820.0, 'lat': 34.8648102777778, 'lon': -86.7702775, 'tz': 'America/Chicago', 'lid': '00AL'}
00AR {'iata': '', 'name': 'Arland Airport', 'city': 'Bennington', 'subd': 'Kansas', 'country': 'US', 'elevation': 1352.0, 'lat': 38.9696510833333, 'lon': -97.6015566111111, 'tz': 'America/Chicago', 'lid': '00AR'}
00AS {'iata': '', 'name': 'Fulton Airport', 'city': 'Alex', 'subd': 'Oklahoma', 'country': 'US', 'elevation': 1100.0, 'lat': 34.9428027777778, 'lon': -97.818

IATA code key

In [12]:
filtered_airport_metadata = {}

for icao, airport_info in airport_metadata.items():
    iata = airport_info['iata']
    country = airport_info['country']
    if iata != '': # only keep airports with lid code
        airport_info.pop('icao', None)  # Remove the 'icao' field
        filtered_airport_metadata[iata] = airport_info

# Print the first 10 entries of the filtered dictionary
for iata, airport_info in list(filtered_airport_metadata.items())[:10]:
    print(iata, airport_info)

# how many airports are there in the filtered_airport_metadata?
n_airports = len(filtered_airport_metadata)
print(f"Number of airports: {n_airports}")

# how many has country 'US'?
n_us_airports = sum(airport_info['country'] == 'US' for airport_info in filtered_airport_metadata.values())
print(f"Number of US airports: {n_us_airports}")

ICY {'iata': 'ICY', 'name': 'Icy Bay Airport', 'city': 'Icy Bay', 'subd': 'Alaska', 'country': 'US', 'elevation': 50.0, 'lat': 59.9690188888889, 'lon': -141.661770277778, 'tz': 'America/Anchorage', 'lid': '19AK'}
HGZ {'iata': 'HGZ', 'name': 'Hog River Airport', 'city': 'Hogatza', 'subd': 'Alaska', 'country': 'US', 'elevation': 534.0, 'lat': 66.1761333333333, 'lon': -155.684816666667, 'tz': 'America/Anchorage', 'lid': '2AK6'}
BYW {'iata': 'BYW', 'name': 'Blakely Island Airport', 'city': 'Blakely Island', 'subd': 'Washington', 'country': 'US', 'elevation': 66.0, 'lat': 48.5789916666667, 'lon': -122.826290833333, 'tz': 'America/Los_Angeles', 'lid': '38WA'}
BDF {'iata': 'BDF', 'name': 'Rinkenberger Airport', 'city': 'Bradford', 'subd': 'Illinois', 'country': 'US', 'elevation': 808.0, 'lat': 41.2308705555556, 'lon': -89.6156516666667, 'tz': 'America/Chicago', 'lid': '3IS8'}
RFK {'iata': 'RFK', 'name': 'Rollang Field', 'city': 'Rolling Fork', 'subd': 'Mississippi', 'country': 'US', 'elevatio

In [13]:
# does filtered_airport_metadata contain iata key 06A? if so, what is the value?
print('A40' in filtered_airport_metadata_lid)


False


In [14]:
# 6. Enrich airports_iata_code with metadata from airport_data
enriched_airports = []
for index, row in airports_iata_code_filtered.iterrows():
    airport_id = row['id']
    iata = row['code']
    metadata = filtered_airport_metadata.get(iata, {})
    enriched_airport = {
        'id': airport_id,
        'code': iata,
        'name': metadata.get('name', None),
        'city': metadata.get('city', None),
        'country': metadata.get('country', None),
        'subd': metadata.get('subd', None),
        'elevation': metadata.get('elevation', None),
        'lat': metadata.get('lat', None),
        'lon': metadata.get('lon', None),
        'tz': metadata.get('tz', None),
    }
    enriched_airports.append(enriched_airport)

# Convert enriched_airports to DataFrame
enriched_airports_df = pd.DataFrame(enriched_airports)

# Display first 5 rows of enriched_airports_df
enriched_airports_df.head(30)

# display all rows where country is US
# enriched_airports_df[enriched_airports_df['country'] == 'US']

,id,code,name,city,country,subd,elevation,lat,lon,tz
0,1,06A,None,None,None,None,NaN,NaN,NaN,None
1,2,08A,None,None,None,None,NaN,NaN,NaN,None
2,3,1G4,None,None,None,None,NaN,NaN,NaN,None
3,4,6B0,None,None,None,None,NaN,NaN,NaN,None
4,5,8F3,None,None,None,None,NaN,NaN,NaN,None
5,6,A02,None,None,None,None,NaN,NaN,NaN,None
6,7,A03,None,None,None,None,NaN,NaN,NaN,None
7,8,A07,None,None,None,None,NaN,NaN,NaN,None
8,9,A12,None,None,None,None,NaN,NaN,NaN,None
9,10,A13,None,None,None,None,NaN,NaN,NaN,None


https://www.bts.gov/topics/airlines-and-airports/world-airport-codes

In [15]:
# Enrich the first 27 rows of enriched_airports_df with metadata from filtered_airport_metadata_lid
for index, row in enriched_airports_df.iloc[:27].iterrows():
    lid = row['code']
    metadata = filtered_airport_metadata_lid.get(lid, {})
    enriched_airports_df.loc[index, 'name'] = metadata.get('name', None)
    enriched_airports_df.loc[index, 'city'] = metadata.get('city', None)
    enriched_airports_df.loc[index, 'country'] = metadata.get('country', None)
    enriched_airports_df.loc[index, 'subd'] = metadata.get('subd', None)
    enriched_airports_df.loc[index, 'elevation'] = metadata.get('elevation', None)
    enriched_airports_df.loc[index, 'lat'] = metadata.get('lat', None)
    enriched_airports_df.loc[index, 'lon'] = metadata.get('lon', None)
    enriched_airports_df.loc[index, 'tz'] = metadata.get('tz', None)

# Display the first 30 rows of enriched_airports_df
enriched_airports_df.head(30)


,id,code,name,city,country,subd,elevation,lat,lon,tz
0,1,06A,Moton Field Municipal Airport,Tuskegee,US,Alabama,264.0,32.460472,-85.680028,America/Chicago
1,2,08A,Wetumpka Municipal Airport,Wetumpka,US,Alabama,197.1,32.527333,-86.331028,America/Chicago
2,3,1G4,Grand Canyon West Airport,Peach Springs,US,Arizona,4816.7,35.986111,-113.816917,America/Phoenix
3,4,6B0,Middlebury State Airport,Middlebury,US,Vermont,494.1,43.986472,-73.095552,America/New_York
4,5,8F3,Crosbyton Municipal Airport,Crosbyton,US,Texas,3018.1,33.623833,-101.240861,America/Chicago
5,6,A02,None,None,None,None,NaN,NaN,NaN,None
6,7,A03,None,None,None,None,NaN,NaN,NaN,None
7,8,A07,None,None,None,None,NaN,NaN,NaN,None
8,9,A12,None,None,None,None,NaN,NaN,NaN,None
9,10,A13,Bold Airport,Anchorage,US,Alaska,900.0,61.341339,-148.998881,America/Anchorage


In [16]:
# add following data for matching iata code: 
# code: A02, city: Deadmans Bay, country: US, subd: AK, elevation: 0, lat: 55.11669921875, lon: -131.56689453125, tz: America/Juneau, name: Deadmans Bay Airport
# iata: A03, city: Hallo Bay, country: US, subd: AK, elevation: 0, lat: 58.89830017089844, lon: -153.18099975585938, tz: America/Anchorage, name: Hallo Bay Airport
# iata: A07, city: Selawik, country: US, subd: AK, elevation: 0, lat: 66.60009765625, lon: -159.98599243164062, tz: America/Nome, name: Roland Norton Memorial
# iata: A12, city: Cinnabar, country: US, subd: MT, elevation: 0, lat: 45.18330001831055, lon: -110.93399810791016, tz: America/Denver, name: Cinnabar Airport
# iata: A23, city: Bradley Lake, country: US, subd: AK, elevation: 0, lat: 60.483299255371094, lon: -151.0330047607422, tz: America/Anchorage, name: Bradley Lake Airport
# iata: A27, city: Pogo Mines, country: US, subd: AK, elevation: 0, lat: 64.8499984741211, lon: -141.14999389648438, tz: America/Yakutat, name: Pogo Mines Airport
# iata: A29, city: Kiluda Bay, country: US, subd: AK, elevation: 0, lat: 59.83330154418945, lon: -151.60000610351562, tz: America/Anchorage, name: Kiluda Bay Airport
# iata: A37, city: Kako, country: US, subd: AK, elevation: 0, lat: 59.83330154418945, lon: -152.58399963378906, tz: America/Anchorage, name: Kako Airport
# iata: A40, city: Cape Simpson, country: US, subd: AK, elevation: 0, lat: 70.7332992553711, lon: -157.14999389648438, tz: America/Nome, name: Cape Simpson Airport
# iata: A52, city: Silver Salmon Creek, country: US, subd: AK, elevation: 0, lat: 59.83330154418945, lon: -153.5, tz: America/Anchorage, name: Silver Salmon Creek Airport
# iata: A71, city: Homer, country: US, subd: AK, elevation: 0, lat: 59.63330078125, lon: -151.5500030517578, tz: America/Anchorage, name: Bear Cove Farm
# iata: A72, city: Chinitna Bay, country: US, subd: AK, elevation: 0, lat: 59.83330154418945, lon: -153.5, tz: America/Anchorage, name: Chinitna Bay Airport
# iata: A74, city: Nanwalek, country: US, subd: AK, elevation: 0, lat: 59.43330001831055, lon: -151.8000030517578, tz: America/Anchorage, name: Dog Fish Bay
# iata: A83, city: Petrof Point, country: US, subd: AK, elevation: 0, lat: 59.83330154418945, lon: -153.5, tz: America/Anchorage, name: Petrof Point Airport
# iata: A97, city: Akulik, country: US, subd: AK, elevation: 0, lat: 59.83330154418945, lon: -153.5, tz: America/Anchorage, name: Akulik Air Strip
# iata: A98, city: Deadfall, country: US, subd: AK, elevation: 0, lat: 59.83330154418945, lon: -153.5, tz: America/Anchorage, name: Deadfall Air Strip
# iata: AA8, city: Deadhorse, country: US, subd: AK, elevation: 0, lat: 70.19999694824219, lon: -148.4669952392578, tz: America/Anchorage, name: Badami

In [17]:
# Data to be added
additional_data = [
    {'code': 'A02', 'city': 'Deadmans Bay', 'country': 'US', 'subd': 'Alaska', 'elevation': 0, 'lat': 55.11669921875, 'lon': -131.56689453125, 'tz': 'America/Juneau', 'name': 'Deadmans Bay Airport'},
    {'code': 'A03', 'city': 'Hallo Bay', 'country': 'US', 'subd': 'Alaska', 'elevation': 0, 'lat': 58.89830017089844, 'lon': -153.18099975585938, 'tz': 'America/Anchorage', 'name': 'Hallo Bay Airport'},
    {'code': 'A07', 'city': 'Selawik', 'country': 'US', 'subd': 'Alaska', 'elevation': 0, 'lat': 66.60009765625, 'lon': -159.98599243164062, 'tz': 'America/Nome', 'name': 'Roland Norton Memorial'},
    {'code': 'A12', 'city': 'Cinnabar', 'country': 'US', 'subd': 'Alaska', 'elevation': 0, 'lat': 45.18330001831055, 'lon': -110.93399810791016, 'tz': 'America/Denver', 'name': 'Cinnabar Airport'},
    {'code': 'A23', 'city': 'Bradley Lake', 'country': 'US', 'subd': 'Alaska', 'elevation': 0, 'lat': 60.483299255371094, 'lon': -151.0330047607422, 'tz': 'America/Anchorage', 'name': 'Bradley Lake Airport'},
    {'code': 'A27', 'city': 'Pogo Mines', 'country': 'US', 'subd': 'Alaska', 'elevation': 0, 'lat': 64.8499984741211, 'lon': -141.14999389648438, 'tz': 'America/Yakutat', 'name': 'Pogo Mines Airport'},
    {'code': 'A29', 'city': 'Kiluda Bay', 'country': 'US', 'subd': 'Alaska', 'elevation': 0, 'lat': 59.83330154418945, 'lon': -151.60000610351562, 'tz': 'America/Anchorage', 'name': 'Kiluda Bay Airport'},
    {'code': 'A37', 'city': 'Kako', 'country': 'US', 'subd': 'Alaska', 'elevation': 0, 'lat': 59.83330154418945, 'lon': -152.58399963378906, 'tz': 'America/Anchorage', 'name': 'Kako Airport'},
    {'code': 'A40', 'city': 'Cape Simpson', 'country': 'US', 'subd': 'Alaska', 'elevation': 0, 'lat': 70.7332992553711, 'lon': -157.14999389648438, 'tz': 'America/Nome', 'name': 'Cape Simpson Airport'},
    {'code': 'A52', 'city': 'Silver Salmon Creek', 'country': 'US', 'subd': 'Alaska', 'elevation': 0, 'lat': 59.83330154418945, 'lon': -153.5, 'tz': 'America/Anchorage', 'name': 'Silver Salmon Creek Airport'},
    {'code': 'A71', 'city': 'Homer', 'country': 'US', 'subd': 'Alaska', 'elevation': 0, 'lat': 59.63330078125, 'lon': -151.5500030517578, 'tz': 'America/Anchorage', 'name': 'Bear Cove Farm'},
    {'code': 'A72', 'city': 'Chinitna Bay', 'country': 'US', 'subd': 'Alaska', 'elevation': 0, 'lat': 59.83330154418945, 'lon': -153.5, 'tz': 'America/Anchorage', 'name': 'Chinitna Bay Airport'},
    {'code': 'A74', 'city': 'Nanwalek', 'country': 'US', 'subd': 'Alaska', 'elevation': 0, 'lat': 59.43330001831055, 'lon': -151.8000030517578, 'tz': 'America/Anchorage', 'name': 'Dog Fish Bay'},
    {'code': 'A83', 'city': 'Petrof Point', 'country': 'US', 'subd': 'Alaska', 'elevation': 0, 'lat': 59.83330154418945, 'lon': -153.5, 'tz': 'America/Anchorage', 'name': 'Petrof Point Airport'},
    {'code': 'A97', 'city': 'Akulik', 'country': 'US', 'subd': 'Alaska', 'elevation': 0, 'lat': 59.83330154418945, 'lon': -153.5, 'tz': 'America/Anchorage', 'name': 'Akulik Air Strip'},
    {'code': 'A98', 'city': 'Deadfall', 'country': 'US', 'subd': 'Alaska', 'elevation': 0, 'lat': 59.83330154418945, 'lon': -153.5, 'tz': 'America/Anchorage', 'name': 'Deadfall Air Strip'},
    {'code': 'AA8', 'city': 'Deadhorse', 'country': 'US', 'subd': 'Alaska', 'elevation': 0, 'lat': 70.19999694824219, 'lon': -148.4669952392578, 'tz': 'America/Anchorage', 'name': 'Badami'},
]

# Add the additional data to enriched_airports_df
for data in additional_data:
    code = data['code']
    matching_index = enriched_airports_df[enriched_airports_df['code'] == code].index
    if not matching_index.empty:
        matching_index = matching_index[0]
        for key, value in data.items():
            enriched_airports_df.loc[matching_index, key] = value

# Display the updated enriched_airports_df
enriched_airports_df.head(50)

,id,code,name,city,country,subd,elevation,lat,lon,tz
0,1,06A,Moton Field Municipal Airport,Tuskegee,US,Alabama,264.0,32.460472,-85.680028,America/Chicago
1,2,08A,Wetumpka Municipal Airport,Wetumpka,US,Alabama,197.1,32.527333,-86.331028,America/Chicago
2,3,1G4,Grand Canyon West Airport,Peach Springs,US,Arizona,4816.7,35.986111,-113.816917,America/Phoenix
3,4,6B0,Middlebury State Airport,Middlebury,US,Vermont,494.1,43.986472,-73.095552,America/New_York
4,5,8F3,Crosbyton Municipal Airport,Crosbyton,US,Texas,3018.1,33.623833,-101.240861,America/Chicago
5,6,A02,Deadmans Bay Airport,Deadmans Bay,US,Alaska,0.0,55.116699,-131.566895,America/Juneau
6,7,A03,Hallo Bay Airport,Hallo Bay,US,Alaska,0.0,58.898300,-153.181000,America/Anchorage
7,8,A07,Roland Norton Memorial,Selawik,US,Alaska,0.0,66.600098,-159.985992,America/Nome
8,9,A12,Cinnabar Airport,Cinnabar,US,Alaska,0.0,45.183300,-110.933998,America/Denver
9,10,A13,Bold Airport,Anchorage,US,Alaska,900.0,61.341339,-148.998881,America/Anchorage
